In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE
from tensorflow.keras import regularizers
###
import tensorflow.keras.utils as ku 
import numpy as np
import keras
from keras import layers
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
tokenizer = Tokenizer()
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

--2021-02-10 20:08:59--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.128, 173.194.217.128, 172.253.123.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2021-02-10 20:08:59 (63.8 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [3]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [4]:

model = keras.models.Sequential()
model.add(Embedding(total_words, 300, input_length=max_sequence_len-1))
model.add(layers.Bidirectional(layers.LSTM(512, return_sequences=True)))
model.add(layers.LSTM(128))
model.add(layers.Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics='accuracy')


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 300)           963300    
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 1024)          3330048   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               590336    
_________________________________________________________________
dense (Dense)                (None, 3211)              414219    
Total params: 5,297,903
Trainable params: 5,297,903
Non-trainable params: 0
_________________________________________________________________


In [6]:
address="./kaggle/working/weights-improvement.hdf5"
stop = EarlyStopping(monitor = 'val_accuracy', min_delta = 0, 
                             patience = 5, verbose = 1, mode = 'auto')
save = ModelCheckpoint(address, monitor = 'val_accuracy', 
                               verbose = 0, save_best_only = True)
callbacks = [stop, save]

history = model.fit(predictors, label, validation_split=0.20, epochs=50, verbose=1, callbacks = callbacks)

Epoch 1/50
387/387 [==============================] - 11s 28ms/step - loss: 6.9505 - accuracy: 0.0234 - val_loss: 6.8310 - val_accuracy: 0.0165
Epoch 2/50
387/387 [==============================] - 9s 24ms/step - loss: 6.4971 - accuracy: 0.0255 - val_loss: 6.9796 - val_accuracy: 0.0213
Epoch 3/50
387/387 [==============================] - 9s 24ms/step - loss: 6.3699 - accuracy: 0.0318 - val_loss: 7.0671 - val_accuracy: 0.0200
Epoch 4/50
387/387 [==============================] - 10s 25ms/step - loss: 6.2679 - accuracy: 0.0400 - val_loss: 7.1536 - val_accuracy: 0.0213
Epoch 5/50
387/387 [==============================] - 9s 24ms/step - loss: 6.1853 - accuracy: 0.0402 - val_loss: 7.2066 - val_accuracy: 0.0220
Epoch 6/50
387/387 [==============================] - 9s 24ms/step - loss: 6.1047 - accuracy: 0.0420 - val_loss: 7.2719 - val_accuracy: 0.0339
Epoch 7/50
387/387 [==============================] - 9s 23ms/step - loss: 6.0301 - accuracy: 0.0463 - val_loss: 7.2107 - val_accuracy: 0.03

Although using val_accuracy do not make much sense in this context, I am using it to get a sense of how good the model architecture really is. The point to remember is that we do not need the exact words while generating the next words for an unknown text. We want words to be similar and appropriate, and most of all interesting to the context provided via a seed text (unknown). Increasing the Embedding dimension brought the training until 26 epochs.